In [1]:
# Required Libraries
import re
import unicodedata
import nltk
import string
from nltk.corpus import stopwords
from collections import Counter
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils import to_categorical

# # Download NLTK stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

#### Section B: Text Preprocessing and Word Embedding (40 Marks)

#### Question 2: Text Preprocessing (20 Marks)

#### Given the dataset reviews.csv, perform the following preprocessing steps:

#### (a) Load the dataset and display the first 5 rows. (5 Marks)

#### (b) Remove any duplicate reviews. (5 Marks)

#### (c) Clean the text by removing punctuation, converting to lowercase, and removing stopwords. (10 Marks)

In [3]:
import pandas as pd
import re
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

df = pd.read_csv('reviews.csv')
print('\n Info', df.info())
print('\nShape', df.shape)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Review     386 non-null    object
 1   Sentiment  386 non-null    object
dtypes: object(2)
memory usage: 6.2+ KB

 Info None

Shape (386, 2)


[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [4]:
df.describe()

,Review,Sentiment
count,386,386
unique,131,3
top,This product exceeded my expectations! It's hi...,Positive
freq,22,129


In [5]:
print(f"\n Top five Rows : {df.head(5)}") 


 Top five Rows :                                               Review Sentiment
0  This product exceeded my expectations! It's hi...  Positive
1  The product was decent. It worked fine, but it...   Neutral
2  I had a terrible experience with this company....  Negative
3  It's an okay product. Nothing to write home ab...   Neutral
4  Disappointed with the product. It didn't meet ...  Negative


In [6]:
df.duplicated().sum()

255

In [7]:
df = df.drop_duplicates(subset = 'Review')

In [8]:

df = df.drop_duplicates(subset='Review')
print("\nShape after remving the duplications:", df.shape)



Shape after remving the duplications: (131, 2)


In [9]:
print(" Converting to lowercase ")
df['Review'] = df['Review'].str.lower()
df['Review'] .head()

 Converting to lowercase 


0    this product exceeded my expectations! it's hi...
1    the product was decent. it worked fine, but it...
2    i had a terrible experience with this company....
3    it's an okay product. nothing to write home ab...
4    disappointed with the product. it didn't meet ...
Name: Review, dtype: object

In [10]:
print('Remove punctuations')
df['Review'] = df['Review'].apply(
    lambda x : ''.join( [ ch if ch not in string.punctuation else '' for ch in x ])
)
df['Review'].head(10)

Remove punctuations


0     this product exceeded my expectations its high...
1     the product was decent it worked fine but it w...
2     i had a terrible experience with this company ...
3       its an okay product nothing to write home about
4     disappointed with the product it didnt meet my...
5     avoid this company at all costs the service is...
9     this product is outstanding its exactly what i...
10            absolutely horrendous service never again
12             this experience was okay nothing special
13    topquality product very satisfied with my purc...
Name: Review, dtype: object

In [11]:
print('Remove stopwords \n')
stop_words = set(stopwords.words('english'))
df['Review'] = df['Review'].apply(
    lambda x: ' '.join([word for word in x.split() if word not in stop_words])
)
print(df['Review'].head(10))


Remove stopwords 

0     product exceeded expectations highquality perf...
1     product decent worked fine wasnt anything special
2     terrible experience company customer service r...
3                       okay product nothing write home
4          disappointed product didnt meet expectations
5     avoid company costs service terrible products ...
9     product outstanding exactly looking works perf...
10                  absolutely horrendous service never
12                      experience okay nothing special
13                topquality product satisfied purchase
Name: Review, dtype: object




from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

df['Review'] = df['Review'].apply(
    lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words])
)

print(df['Review'].head())

#### Question 3: Word Embedding (20 Marks)

#### (a) Convert the cleaned text into word embeddings using TF-IDF(10 Marks)

#### (b) Display the shape of the resulting TF-IDF matrix. (4 Marks)

#### (c) Get the vocabulary of TF-IDF  (6 marks)



#### Solution:

In [12]:
# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Review'])
print('TF-IDF matrix shape',X.shape)
vocab = vectorizer.get_feature_names_out()
print("TF-IDF Vocabulary Size:", len(vocab))
print("Sample Vocabulary Words:", vocab[:20])


TF-IDF matrix shape (131, 162)
TF-IDF Vocabulary Size: 162
Sample Vocabulary Words: ['absolutely' 'advertised' 'amazed' 'amazing' 'anyone' 'anything'
 'arrived' 'atrocious' 'attention' 'average' 'avoid' 'away' 'awful' 'back'
 'bad' 'better' 'beyond' 'blow' 'bought' 'brilliant']


### Section B: Model Building for Sentiment Analysis (40 Marks)

#### Question 4: Naive Bayes Classifier (20 Marks)

#### (a) Split the dataset into training and testing sets. (5 Marks)

#### (b) Train a Naive Bayes classifier on the training set. (10 Marks)

#### (c) Evaluate the model on the testing set and display the accuracy. (5 Marks)



In [13]:
# (a) Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


y = df['Sentiment']
# Target labels (ensure this column exists)

# Split data (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# (b) Train Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train, y_train)

# (c) Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Naive Bayes Model Accuracy:", round(accuracy * 100, 2), "%")



Naive Bayes Model Accuracy: 96.3 %


#### Question 5: LSTM Model (20 Marks)

#### (a) Convert the cleaned text into sequences using Tokenizer. (5 Marks)

#### (b) Build and compile an LSTM model for sentiment analysis. (10 Marks)

#### (c) Train the model and evaluate its performance on the testing set. (5 Marks)



In [24]:
from tensorflow.keras.layers import Dropout

# (a) Convert cleaned text into sequences using Tokenizer
max_words = 5000  # Vocabulary size
max_len = 100     # Max length of each sequence

tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(df['Review'])
sequences = tokenizer.texts_to_sequences(df['Review'])
X = pad_sequences(sequences, padding='post')

# Label encoding (Assuming Sentiment is 'positive'/'negative')
le = LabelEncoder()
y = le.fit_transform(df['Sentiment'])
y = to_categorical(y)

# Split the dataset
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    Embedding(input_dim = 10000, output_dim = 128),
    LSTM(  64, return_sequences=False),
    Dropout(0.3),
    Dense(64,activation='relu'),
    Dense(y.shape[1],activation='softmax')

])

model.build((None, X.shape[1]))
model.summary()

model.compile(optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

model.fit( x_train , y_train ,batch_size=32,
    epochs=10,validation_data=(x_test, y_test))


loss , acc = model.evaluate(x_test, y_test)

print ( 'accuracy is ',acc * 100)
print ( 'loss  is ',loss * 100)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ (None, 12, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,333,763 (5.09 MB)

 Trainable params: 1,333,763 (5.09 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 121ms/step - accuracy: 0.4293 - loss: 1.0975 - val_accuracy: 0.3333 - val_loss: 1.0940
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.3728 - loss: 1.0988 - val_accuracy: 0.3333 - val_loss: 1.0948
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.3587 - loss: 1.0953 - val_accuracy: 0.4074 - val_loss: 1.0907
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.4174 - loss: 1.0933 - val_accuracy: 0.5556 - val_loss: 1.0863
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.4604 - loss: 1.0842 - val_accuracy: 0.5926 - val_loss: 1.0761
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.4648 - loss: 1.0737 - val_accuracy: 0.3333 - val_loss: 1.0572
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.4759 - loss: 1.0434 - val_accuracy: 0.4444 - val_loss: 1.0173
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.6623 - loss: 0.9867 - val_accuracy: 0.8889 - val_loss: 0.9300

In [ ]:
dir(Tokenizer)